In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Monte Carlo Integration - Complete Assignment Solution\n",
    "\n",
    "**Student: IIT Bombay CSE 2nd Year**\n",
    "*All methods fully vectorized, convergence plots, high-dim demo*"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "from scipy.special import erf\n",
    "np.random.seed(42)\n",
    "plt.rcParams['figure.figsize'] = (12, 8)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## **1. π Estimation (Assignment 2.1)**"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "class PiCalc:\n",
    "    def __init__(self):\n",
    "        self.actual = np.pi\n",
    "        \n",
    "    def run(self, n):\n",
    "        x = np.random.uniform(-1, 1, n)\n",
    "        y = np.random.uniform(-1, 1, n)\n",
    "        hits = np.sum(x**2 + y**2 <= 1)\n",
    "        return 4 * hits / n\n",
    "    \n",
    "    def check_trend(self):\n",
    "        sizes = [10 * 2**i for i in range(18)]\n",
    "        vals = [self.run(s) for s in sizes]\n",
    "        errs = 100 * np.abs(np.array(vals) - self.actual) / self.actual\n",
    "        \n",
    "        fig, (p1, p2) = plt.subplots(1, 2, figsize=(12, 5))\n",
    "        p1.semilogx(sizes, vals, 'bo-', linewidth=2, markersize=6)\n",
    "        p1.axhline(self.actual, color='r', ls='--', linewidth=2)\n",
    "        p1.set_ylabel('Estimate')\n",
    "        p1.grid(True, alpha=0.3)\n",
    "        p1.set_title('π Convergence')\n",
    "        \n",
    "        p2.loglog(sizes, errs, 'ro-', linewidth=2, markersize=6)\n",
    "        p2.set_xlabel('N (log)')\n",
    "        p2.set_ylabel('Error % (log)')\n",
    "        p2.grid(True, alpha=0.3)\n",
    "        \n",
    "        plt.tight_layout()\n",
    "        plt.savefig('pi_convergence.png', dpi=300, bbox_inches='tight')\n",
    "        plt.show()\n",
    "        return sizes[-1], vals[-1], errs[-1]\n",
    "\n",
    "pi = PiCalc()\n",
    "print('Final π:', pi.check_trend())"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## **2. e Estimation - Both Methods (Assignment 2.2)**"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "class ECalc:\n",
    "    def __init__(self):\n",
    "        self.actual = np.e\n",
    "        \n",
    "    def way1(self, n, upper=2):\n",
    "        x = np.random.uniform(1, upper, n)\n",
    "        y = np.random.uniform(0, 1, n)\n",
    "        under = np.sum(y < 1/x)\n",
    "        area = (upper-1) * under / n\n",
    "        return upper ** (1/area) if area > 0 else np.nan\n",
    "    \n",
    "    def way2(self, runs):  # Vectorized Forsythe method\n",
    "        sz = 25\n",
    "        rnd = np.random.uniform(0, 1, (runs, sz))\n",
    "        diffs = np.diff(rnd, axis=1)\n",
    "        breaks = np.where(diffs >= 0, np.arange(1, sz), sz-1)\n",
    "        breaks = np.min(breaks, axis=1) + 2\n",
    "        return np.mean(breaks)\n",
    "    \n",
    "    def compare_methods(self):\n",
    "        counts = [10 * 2**i for i in range(16)]\n",
    "        m1 = [self.way1(c) for c in counts]\n",
    "        m2 = [self.way2(c) for c in counts]\n",
    "        \n",
    "        fig, (g1, g2) = plt.subplots(1, 2, figsize=(12, 5))\n",
    "        g1.semilogx(counts, m1, 'b.-', linewidth=2, markersize=8, label='Area Method')\n",
    "        g1.semilogx(counts, m2, 'g.-', linewidth=2, markersize=8, label='Chain Method')\n",
    "        g1.axhline(self.actual, color='r', ls='--', linewidth=2, label='True e')\n",
    "        g1.legend()\n",
    "        g1.grid(True, alpha=0.3)\n",
    "        g1.set_ylabel('Estimate')\n",
    "        g1.set_title('e Estimation Comparison')\n",
    "        \n",
    "        e1 = 100 * np.abs(np.array(m1) - self.actual) / self.actual\n",
    "        e2 = 100 * np.abs(np.array(m2) - self.actual) / self.actual\n",
    "        g2.loglog(counts, e1, 'b.-', linewidth=2, markersize=8)\n",
    "        g2.loglog(counts, e2, 'g.-', linewidth=2, markersize=8)\n",
    "        g2.set_xlabel('N (log)')\n",
    "        g2.set_ylabel('Error % (log)')\n",
    "        g2.grid(True, alpha=0.3)\n",
    "        g2.legend(['Area', 'Chain'])\n",
    "        \n",
    "        plt.tight_layout()\n",
    "        plt.savefig('e_methods.png', dpi=300, bbox_inches='tight')\n",
    "        plt.show()\n",
    "        \n",
    "e = ECalc()\n",
    "print(f\"e method1: {e.way1(100000):.5f}\")\n",
    "print(f\"e method2: {e.way2(10000):.5f}\")\n",
    "print(f\"true e:    {np.e:.5f}\")\n",
    "e.compare_methods()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## **3. General Monte Carlo Shapes (Assignment 2.3)**"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "class AreaSampler:\n",
    "    def __init__(self, shape_func, bounds, true_val=None):\n",
    "        self.shape_func = shape_func\n",
    "        self.bounds = bounds\n",
    "        self.true_val = true_val\n",
    "    \n",
    "    def estimate(self, N):\n",
    "        xmin, xmax, ymin, ymax = self.bounds\n",
    "        x = np.random.uniform(xmin, xmax, N)\n",
    "        y = np.random.uniform(ymin, ymax, N)\n",
    "        inside = self.shape_func(x, y)\n",
    "        ratio = np.sum(inside) / N\n",
    "        box_area = (xmax-xmin) * (ymax-ymin)\n",
    "        return box_area * ratio\n",
    "    \n",
    "    def convergence(self, values):\n",
    "        results, errors = [], []\n",
    "        for value in values:\n",
    "            est = self.estimate(value)\n",
    "            results.append(est)\n",
    "            if self.true_val:\n",
    "                err = 100 * np.abs(est - self.true_val) / self.true_val\n",
    "                errors.append(err)\n",
    "        return results, errors\n",
    "\n",
    "# Define shapes\n",
    "circle = lambda x,y: x**2 + y**2 <= 1\n",
    "parabola = lambda x,y: y <= x**2\n",
    "gaussian = lambda x,y: y <= np.exp(-x**2)\n",
    "\n",
    "# Setup samplers with true values\n",
    "samplers = {\n",
    "    'Circle': AreaSampler(circle, [-1,1,-1,1], np.pi),\n",
    "    'Parabola': AreaSampler(parabola, [0,1,0,1], 1/3),\n",
    "    'Gaussian': AreaSampler(gaussian, [0,2,0,1], np.sqrt(np.pi)/2 * erf(2))\n",
    "}\n",
    "\n",
    "# Test single estimates\n",
    "N = 1000000\n",
    "for name, sampler in samplers.items():\n",
    "    est = sampler.estimate(N)\n",
    "    true = sampler.true_val if sampler.true_val else 'N/A'\n",
    "    print(f'{name}: {est:.5f} {\"|\" if true else \"\"} {true:.5f}')"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## **4. Convergence Analysis (Assignment 2.4)**"
   ]
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "N_values = [1000, 10_000, 100_000, 1_000_000]\n",
    "\n",
    "fig, axes = plt.subplots(2, 2, figsize=(12, 10))\n",
    "axes = axes.flatten()\n",
    "\n",
    "for idx, (name, sampler) in enumerate(samplers.items()):\n",
    "    results, errors = sampler.convergence(N_values)\n",
    "    \n",
    "    axes[idx].loglog(N_values, errors, 'o-', linewidth=3, markersize=8)\n",
    "    axes[idx].set_title(f'{name} Error Convergence')\n",
    "    axes[idx].set_xlabel('N')\n",
    "    axes[idx].set_ylabel('Error %')\n",
    "    axes[idx].grid(True, alpha=0.3)\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.savefig('shapes_convergence.png', dpi=300, bbox_inches='tight')\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## **5. High-Dimensional Demo (Bonus)**"
   ]
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def unit_ball_volume(dims, N=100_000):\n",
    "    points = np.random.uniform(-1, 1, (N, dims))\n",
    "    inside = np.sum(points**2, axis=1) <= 1\n",
    "    vol = np.prod([2]*dims) * np.mean(inside)\n",
    "    return vol\n",
    "\n",
    "dims = [2, 4, 8, 16]\n",
    "results = [unit_ball_volume(d) for d in dims]\n",
    "\n",
    "plt.figure(figsize=(10, 6))\n",
    "plt.semilogy(dims, results, 'ro-', linewidth=3, markersize=10)\n",
    "plt.xlabel('Dimensions')\n",
    "plt.ylabel('Estimated Volume')\n",
    "plt.title('Unit Ball Volume Curse of Dimensionality')\n",
    "plt.grid(True, alpha=0.3)\n",
    "plt.savefig('high_dim.png', dpi=300, bbox_inches='tight')\n",
    "plt.show()\n",
    "\n",
    "print('High-dim volumes:', results)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## **Summary Table**\n",
    "\n",
    "| Method | N=1M Estimate | True Value | Error % |\n",
    "|--------|---------------|------------|---------|\n",
    "| π | [FILL] | 3.14159 | [FILL] |\n",
    "| e (Area) | [FILL] | 2.71828 | [FILL] |\n",
    "| Circle | [FILL] | 3.14159 | [FILL] |\n",
    "| Parabola | [FILL] | 0.33333 | [FILL] |\n",
    "| Gaussian | [FILL] | [FILL] | [FILL] |"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}
